In [ ]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
c
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive



Mounted at /gdrive
/gdrive


In [ ]:
trainfile = r'/gdrive/My Drive/CIS 508/Assignment 2/Portugese Bank Data - TRAIN.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/My Drive/CIS 508/Assignment 2/Portugese Bank Data - TEST.csv'
testData = pd.read_csv(testfile)  #creates a dataframe


print(trainData.shape)
print(testData.shape)

trainData.head()

(4521, 17)
(45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [ ]:
#List of Categorical Features
categoricalFeatures = ["job", "marital", 'education', "default", "housing", "loan", 'contact', 'month','poutcome']

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData,testData], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)

#Separate Train data and test data
trainData = combined_Data.xs(0)
testData = combined_Data.xs(1)

y_train = trainData["y"]
X_train = trainData.drop(["y"], axis=1) #extracting training data without the target column
y_test = testData["y"]
X_test = testData.drop(["y"], axis=1) #extracting training data without the target column



In [ ]:
#Decision Tree Classifier ========================================================================
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)
print("accuracy Score (testset) for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print()
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))



accuracy Score (testset) for Decision Tree:0.881777

Confusion Matrix for Decision Tree
[[37114  2808]
 [ 2537  2752]]


In [ ]:
#Hyperparameter tuning done for decision tree classifier
#do random search
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(10,100,10),'max_depth': range(5,30,5),'criterion':['gini','entropy']}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15,cv=5)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)
#Now do grid search
print("GridSearchCV-Decision tree")
clf_grid = GridSearchCV(clf,parameters)
clf_grid.fit(X_train, y_train)
grid_parm1=clf_grid.best_params_
print(grid_parm1)



RandomizedSearchCV-Decision tree
{'min_samples_leaf': 60, 'max_depth': 5, 'criterion': 'gini'}
GridSearchCV-Decision tree
{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 30}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier
#Construct Decision Trees using the best parameters
clf = DecisionTreeClassifier(min_samples_leaf= 60, max_depth= 5, criterion= 'gini')
clfr = DecisionTreeClassifier(criterion= 'entropy', max_depth= 5, min_samples_leaf= 30)

clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)
clfr.fit(X_train,y_train)
clfr_predict = clfr.predict(X_test)




In [ ]:
#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (testset) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print()
print("accuracy Score (testset) after hypertuning grid search for Decision Tree:{0:6f}".format(clfr.score(X_test,y_test)))
print()
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print()
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))
print()
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')




accuracy Score (testset) after hypertuning randomized search for Decision Tree:0.896596

accuracy Score (testset) after hypertuning grid search for Decision Tree:0.896220

Confusion Matrix after hypertuning for Decision Tree
[[38265  1657]
 [ 3018  2271]]

=== Classification Report ===
              precision    recall  f1-score   support

          no       0.93      0.96      0.94     39922
         yes       0.58      0.43      0.49      5289

    accuracy                           0.90     45211
   macro avg       0.75      0.69      0.72     45211
weighted avg       0.89      0.90      0.89     45211


[0.64537736 0.65346154 0.66269231 0.65096154 0.67567308 0.62336538
 0.66980769 0.64009615 0.65346154 0.68903846]




In [ ]:
#Normal randomforest==============================================================================
#=================================================================================================
rand_parameters={'min_samples_leaf' : range(10,100,10),'max_depth': range(1,10,2),'max_features':[2,3,4],'n_estimators':[20,30,40]}
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
print("accuracy Score (testset) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
#do random search with cross-validation
rfc_random = RandomizedSearchCV(rfc,rand_parameters,n_iter=15,cv=5)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)
#create new classifier using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
print("accuracy Score (testset) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))
rfc_cv_score = cross_val_score(rfc, X_train, y_train)
print(rfc_cv_score)
print('\n')



accuracy Score (testset) for RandomForest:0.912057
Confusion Matrix for Random Forest:
[[39243   679]
 [ 3297  1992]]
{'n_estimators': 30, 'min_samples_leaf': 20, 'max_features': 4, 'max_depth': 9}
accuracy Score (testset) after hypertuning for Random Forest:0.886045
Confusion Matrix after hypertuning for Random Forest:
[[39892    30]
 [ 5122   167]]
=== Classification Report ===
              precision    recall  f1-score   support

          no       0.89      1.00      0.94     39922
         yes       0.85      0.03      0.06      5289

    accuracy                           0.89     45211
   macro avg       0.87      0.52      0.50     45211
weighted avg       0.88      0.89      0.84     45211

[0.88839779 0.88495575 0.88495575 0.88495575 0.88495575]




In [ ]:
#Gradient Boosting================================================================================
search_grid={'n_estimators':[5,10,20],'learning_rate':[0.01,.1,.2],'min_samples_leaf' : range(10,100,10),'max_depth': range(1,10,2)}
abc =GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_test)
print("accuracy Score (testset) for Boosting:{0:6f}".format(abc.score(X_test,y_test)))
print("Confusion Matrix for boosting:")
print(confusion_matrix(y_test,abc_predict))
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,y_train)
abc_predict = abc.predict(X_test)
print("accuracy Score (testset) after hypertuning for Boosting:{0:6f}".format(abc.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Boosting:")
print(confusion_matrix(y_test,abc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,abc_predict))
abc_cv_score = cross_val_score(abc, X_train, y_train)
print(abc_cv_score)
print('\n')


accuracy Score (testset) for Boosting:0.647500
Confusion Matrix for boosting:
[[ 98  16  27]
 [ 20  52  34]
 [ 25  19 109]]
{'n_estimators': 10, 'min_samples_leaf': 70, 'max_depth': 5, 'learning_rate': 0.1}
accuracy Score (testset) after hypertuning for Boosting:0.565000
Confusion Matrix after hypertuning for Boosting:
[[ 97   3  41]
 [ 38  25  43]
 [ 38  11 104]]
=== Classification Report ===
              precision    recall  f1-score   support

           1       0.56      0.69      0.62       141
           2       0.64      0.24      0.34       106
           3       0.55      0.68      0.61       153

    accuracy                           0.56       400
   macro avg       0.58      0.53      0.52       400
weighted avg       0.58      0.56      0.54       400

[0.54576271 0.5220339  0.56610169 0.60544218 0.54761905]


